<a href="https://colab.research.google.com/github/katya-stanzy/python_work/blob/main/TextAnalysisRegex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Text Analysis

## What kinds of text analysis are there?

* analyst knows the pattern
    * regular expressions
* analyst does not know the pattern
    * natural language processing
        * compares historical examples to judge novel cases
            * comparisons are statistical and approximate
            

### Examples of Analysis

When you know the pattern:

In [ ]:
pattern = '£ ?[0-9][0-9]?' # £ then SPACE-optional then digit then digit-optional 

document = 'My eggs cost £3, bread cost £2, vodka cost £35'

In [ ]:
import re

In [ ]:
re.findall(pattern, document)

If you dont:

* sentiment analysis
    * how positive/negative is this (new) review?
* topic analysis 
    * what is this document about?

## What can I do if I know what pattern I want to find?

* finding ("extracting")
    * what matches the pattern?
* matching ("validating")
    * does the entire document match YES/NO?
* substitue ("replacing")
    * replace a part that matches a pattern with another...

## How do I validate text with pandas?

In [1]:
import pandas as pd

In [5]:
ti = pd.read_csv('datasets/titanic.csv')
ti.sample(1)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
498,0,1,female,25.0,1,2,151.55,S,First,woman,False,C,Southampton,no,False


In [6]:
ti['ticket'] = "Ticket: " + ti['class'] + "; Price: $ " + ti['fare'].astype(str) + "; Port: " + ti['embark_town'] + ";"

In [7]:
ti[['class', 'fare', 'embark_town', 'ticket']].head(3)

,class,fare,embark_town,ticket
0,Third,7.2500,Southampton,Ticket: Third; Price: $ 7.25; Port: Southampton;
1,First,71.2833,Cherbourg,Ticket: First; Price: $ 71.2833; Port: Cherbourg;
2,Third,7.9250,Southampton,Ticket: Third; Price: $ 7.925; Port: Southampton;


In [8]:
pattern = '(First|Second)'

ti['class'].str.match(pattern)

0      False
1       True
2      False
3       True
4      False
       ...  
886     True
887     True
888    False
889     True
890    False
Name: class, Length: 891, dtype: bool

In [9]:
ti.loc[ ti['class'].str.match(pattern)  , 'survived'].mean()

0.5575

## How do I extract data with pandas?

In [10]:
ti[['class', 'fare', 'embark_town', 'ticket']].head(3)

,class,fare,embark_town,ticket
0,Third,7.2500,Southampton,Ticket: Third; Price: $ 7.25; Port: Southampton;
1,First,71.2833,Cherbourg,Ticket: First; Price: $ 71.2833; Port: Cherbourg;
2,Third,7.9250,Southampton,Ticket: Third; Price: $ 7.925; Port: Southampton;


In [11]:
pattern = '([0-9.]+)'

ti['ticket'].str.extract(pattern).sample(4)

,0
38,18.0
662,25.5875
93,20.575
153,14.5


## How do I substitue text with pandas?

In [12]:
ti['ticket'].str.replace('$', '€').sample(1)

122    Ticket: Second; Price: € 30.0708; Port: Cherbo...
Name: ticket, dtype: object

## What are regular expressions?

Regular expressions are a language for describing patterns in text. 

They are separate from python, but may be used within python program. (And elsewhere, eg., often in SQL). 

They are notoriously difficult to read and write; and as a separate language, an additional tool to learn. 

## What regular expression patterns can I use?

* literals
    * `a`, find me an `a`
    * `£`, find `£`
    * `!` means `!` 
    * ... most symbols mean "find me"
* `.`
    * find any **single** symbol 
* character classes -- find a **single** symbol
    * `[abc]` $\rightarrow$ **either** a, b, c
    * `[0-9]` $\rightarrow$ **either** 0, 1, 2, 3,...9
    * `[A-Z]` $\rightarrow$ **either** capital A, B, ... Z
    * inversions
        * `[^abc]` $\rightarrow$ **is not** `a` OR `b` OR `c`
        * `[^a-zA-Z0-9 ]`  $\rightarrow$ **is not** alphanumeric-ish
    
* alternatives -- find the character**s** given by...
    * `(May|June|July)`  $\rightarrow$ **the whole worlds** May OR June..
    

In [13]:
ti['ticket'].str.extract('(Ticket: (First|Second))')

,0,1
0,NaN,NaN
1,Ticket: First,First
2,NaN,NaN
3,Ticket: First,First
4,NaN,NaN
...,...,...
886,Ticket: Second,Second
887,Ticket: First,First
888,NaN,NaN
889,Ticket: First,First


In [14]:
ti['ticket'].str.extract('( [0-9][0-9])')

,0
0,NaN
1,71
2,NaN
3,53
4,NaN
...,...
886,13
887,30
888,23
889,30


In [16]:
ti['ticket'].str.extract('( [0-9])')

,0
0,7
1,7
2,7
3,5
4,8
...,...
886,1
887,3
888,2
889,3


In [21]:
ti['ticket'].str.extract('( [0-9][0-9][0-9])').dropna()

,0
27,263
31,146
88,263
118,247
195,146
215,113
258,512
268,153
269,135
297,151


In [15]:
ti['ticket'].sample(1)

251    Ticket: Third; Price: $ 10.4625; Port: Southam...
Name: ticket, dtype: object

In [22]:
ti['ticket'].str.extract('(Ticket: [A-Z])').sample(2)

,0
404,Ticket: T
217,Ticket: S


In [23]:
ti['ticket'].str.extract('(T........)').sample(3)

,0
743,Ticket: T
785,Ticket: T
326,Ticket: T


In [24]:
ti['ticket'].str.extract('(Price: [^0-9A-Za-z] ..)').sample(3)

,0
121,Price: $ 8.
622,Price: $ 15
653,Price: $ 7.


In [25]:
ti['ticket'].str.extract('(Port: (Cherbourg|Southampton))').sample(3)

,0,1
8,Port: Southampton,Southampton
786,Port: Southampton,Southampton
626,NaN,NaN


* repetitions
    * optional `?`
        * an optional number: `[0-9]?`
    * one or more `+`
        * one or more spaces: ` +`  
    * optional, or more, `*`
        * ` [0-9][0-9]?.[0-9]*`
    

In [26]:
ti['ticket'].str.extract('([0-9][0-9]?.[0-9]*)').sample(3)

,0
644,19.2583
73,14.4542
19,7.225


In [27]:
ti['ticket'].str.extract('(Ticket: [a-zA-Z]+)').sample(3)

,0
866,Ticket: Second
371,Ticket: Third
598,Ticket: Third


In [28]:
row = 0
match = 1 # second match

ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+)').loc[row, match]

0    Port: Southampton
Name: (0, 1), dtype: object

In [29]:
ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+)') #this gives more than one match for a record; adding loc[row,match] specifies if we want first or second match

0
    match                   
0   0          Ticket: Third
    1      Port: Southampton
1   0          Ticket: First
    1        Port: Cherbourg
2   0          Ticket: Third
...                      ...
888 1      Port: Southampton
889 0          Ticket: First
    1        Port: Cherbourg
890 0          Ticket: Third
    1       Port: Queenstown

[1778 rows x 1 columns]

In [30]:
ti['ticket'].str.extract('([a-zA-Z]+tow?n)')

,0
0,Southampton
1,NaN
2,Southampton
3,Southampton
4,Southampton
...,...
886,Southampton
887,Southampton
888,Southampton
889,NaN


* EXTRA: 
    * escaping
        * How do I say, literally, the `.` symbol?
        * `\.`
    

In [31]:
ti['ticket'].str.extract('(\$ [0-9]+\.[0-9]+)').sample(2)

,0
468,$ 7.725
313,$ 7.8958


* positional matching
    * `^` means **at the beginning**
    * `$` means **at the end**

In [33]:
ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+;$)').sample(4)

,,0
,match,
378,0,Port: Cherbourg;
822,0,Port: Southampton;
810,0,Port: Southampton;
470,0,Port: Southampton;


In [35]:
ti['ticket'].str.extractall('(^[a-zA-Z]+: [a-zA-Z]+;)').sample(4)# the difference between this line and the line above is that $ sign asks for the last symbol in the line and its abscence returns first, even if we ask extractall

,,0
,match,
306,0,Ticket: First;
65,0,Ticket: Third;
470,0,Ticket: Third;
732,0,Ticket: Second;


## Next Steps

* review a "Regex Cheat Sheet"
    * also, eg., https://en.wikipedia.org/wiki/Regular_expression#Examples

## Exercise (30 min)

* find all the words in the tickets 
    * HINT: a word is a repeated letter followed by a space or a colon
    * HINT: `[ :]` means a space or a colon
* find all the USD prices
    * HINT: ``` \$ ``` and repeated numbers 
    
* find all the high-price tickets
    * HINT: consider `\$`, tripple-digit number, `\.`

In [38]:
#Solution

In [39]:
ex1=ti['ticket'].str.findall('([a-zA-Z]+[ :])').sample(10)

In [41]:
type(ex1)

pandas.core.series.Series

In [44]:
ex1

780    [Ticket:, Price:, Port:]
263    [Ticket:, Price:, Port:]
543    [Ticket:, Price:, Port:]
488    [Ticket:, Price:, Port:]
837    [Ticket:, Price:, Port:]
513    [Ticket:, Price:, Port:]
47     [Ticket:, Price:, Port:]
823    [Ticket:, Price:, Port:]
542    [Ticket:, Price:, Port:]
462    [Ticket:, Price:, Port:]
Name: ticket, dtype: object

In [42]:
ex2=ti['ticket'].str.extract('(\$ [0-9]+\.[0-9]+)').sample(2)

In [43]:
type(ex2)

pandas.core.frame.DataFrame

In [45]:
ex2

,0
302,$ 0.0
657,$ 15.5


In [52]:
row = 27
match = 1 # second match
ti['ticket'].str.extractall('(\$ [0-9][0-9][0-9]+\.[0-9]+)').loc[row, 0]

0    $ 263.0
Name: (27, 0), dtype: object

In [53]:
ti['ticket'].str.extractall('(\$ [0-9][0-9][0-9]+\.[0-9]+)')#.loc[row, 0]

,,0
,match,
27,0,$ 263.0
31,0,$ 146.5208
88,0,$ 263.0
118,0,$ 247.5208
195,0,$ 146.5208
215,0,$ 113.275
258,0,$ 512.3292
268,0,$ 153.4625
269,0,$ 135.6333
